In [6]:
%run './style-change-detection.py'

[nltk_data] Downloading package punkt to /home/robert/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/robert/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!

KeyboardInterrupt
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "/home/robert/envs/style-change-detection/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2761, in safe_execfile
    py3compat.execfile(
  File "/home/robert/envs/style-change-detection/lib/python3.8/site-packages/IPython/utils/py3compat.py", line 168, in execfile
    exec(compiler(f.read(), fname, 'exec'), glob, loc)
  File "/home/robert/projects/style-change-detection/style-change-detection.py", line 274, in <module>
    fasttext.util.download_model('en', if_exists='ignore')  # English
  File 

TypeError: object of type 'NoneType' has no len()